In [1]:
from numpy import *
import scipy
from scipy.special import erf

# Valores iniciales

In [2]:
#Diccionario de elementos
#Valor zeta   Núm.cuánticos_máx   Carga
elementos = {'H': {'Z': 1.24, 'Cmax': 1, 'Carga': 1}, 
             'He': {'Z': 2.0925, 'Cmax': 1, 'Carga': 2}}
#Número de gausianas utilizadas
STOnG = 3
#Coeficientes de contracción
D = array([[0.444635, 0.535328, 0.154329],[0.700115,0.399513,-0.0999672]])
#Exponentes de orbitales
α = array([[0.109818, 0.405771, 2.22766],[0.0751386,0.231031,0.994203]])
#Tamaño de conjunto base
B = 2 #La suma de los números cuánticos máximos como usaremos H2 y HeH sólo puede ser '2'
#Número de electrones
N = 2
#Posición de los átomos por molecula

# Definimos las funciones que se utilizarán más adelante

In [3]:
# Se tienen que hacer las integrales entre orbitales Gaussianos
# Esta función hace el producto entre dos gausianas
def producto_gaussiano(gaussα, gaussβ):
    #sólo necesitamos conocer el centro y el exponente para esto
    α, Cα = gaussα
    β, Cβ = gaussβ
    ρ = α + β
    dif = linalg.norm(Cα-Cβ)**2   #Distancia al cuadrado de un centro al otro
    N = (4*α*β/(pi**2))**0.75           #Normalización
    K = N*exp(-α*β/ρ*dif)               #Prefactor nuevo
    Cρ = (α*Cα + β*Cβ)/ρ                #Centro nuevo
    return ρ, dif, K, Cρ

def traslape(A, B):   #Integral de traslape
    ρ, dif, K, Cρ = producto_gaussiano(A, B)
    prefactor = (pi/ρ)**1.5
    return prefactor*K

def cinetica(A, B):      #Integral cinética
    ρ, dif, K, Cρ = producto_gaussiano(A, B)
    prefactor = (pi/ρ)**1.5
    α, Cα = A
    β, Cβ = B
    exponente_reducido = α*β/ρ
    return exponente_reducido*(3-2*exponente_reducido*dif)*prefactor*K

def Fo(t):  #Aquí se calcularan las integrales de repulsión entre electrones y el potencial
    if t == 0:
        return 1
    else:
        return (0.5*(pi/t)**0.5)*erf(t**0.5)
    
def potencial(A, B, atomo):
    ρ, dif, K, Cρ = producto_gaussiano(A, B)
    Cγ = mol[f][1]            #Posición del átomo
    Zγ = elementos[atomo]['Carga']     #Carga del átomo
    return (-2*pi*Zγ/ρ)*K*Fo(ρ*linalg.norm(Cρ-Cγ)**2)
     
def multi(A, B, C, D):
    ρ1, dif1, K1, Cρ1 = producto_gaussiano(A, B)
    ρ2, dif2, K2, Cρ2 = producto_gaussiano(C, D)
    multi_prefactor = 2*pi**2.5*(ρ1*ρ2*(ρ1+ρ2)**0.5)**-1
    return multi_prefactor*K1*K2*Fo(ρ1*ρ2/(ρ1+ρ2)*linalg.norm(Cρ1-Cρ2)**2)

# Sabemos que este cálculo se realiza con matrices, así que las iniciaremos

In [4]:
S = zeros((B,B))
T = zeros((B,B))
V = zeros((B,B))
tensor_multi_electronico = zeros((B,B,B,B))

n = input("Escribe La molécula a calcular 'H2' o 'HeH':")

if n=='HeH':
    mol = [['He', array([0.,0.,0.])], ['H' , array([0.,0.,1.4632])]]
elif n == 'H2':
    mol = [['H', array([0.,0.,0.])], ['H', array([0.,0.,0.74])]]
else:
    print('No se tiene información')

for i in range(len(mol)):
    Za = elementos[mol[i][0]]['Carga']
    Ca = mol[i][1]
    for m in range(elementos[mol[i][0]]['Cmax']):
        d_vec_m = D[m]
        z = elementos[mol[i][0]]['Z']

        α_vec_m = α[m]*z**2

        for p in range(STOnG):
            for j in range(len(mol)):
                Zb = elementos[mol[j][0]]['Carga']
                Cb = mol[j][1]
                for n in range(elementos[mol[j][0]]['Cmax']):
                    d_vec_n = D[n]
                    z = elementos[mol[j][0]]['Z']

                    α_vec_n = α[n]*z**2
                    for q in range(STOnG):
                        a = (i+1)*(m+1)-1
                        b = (j+1)*(n+1)-1

                        S[a,b] += d_vec_m[p]*d_vec_n[q]*traslape((α_vec_m[p],Ca),(α_vec_n[q],Cb))

                        T[a,b] += d_vec_m[p]*d_vec_n[q]*cinetica((α_vec_m[p],Ca),(α_vec_n[q],Cb))
                        
                        f = 0
                        for k in [row[0] for row in mol]:
                            V[a,b] += d_vec_m[p]*d_vec_n[q]*potencial((α_vec_m[p],Ca),(α_vec_n[q],Cb),k)
                            f += 1
                        for l in range(len(mol)):
                            Zc = elementos[mol[l][0]]['Carga']
                            Cc = mol[l][1]
                            for o in range(elementos[mol[l][0]]['Cmax']):
                                d_vec_o = D[o]
                                z = elementos[mol[l][0]]['Z']
                                α_vec_o = α[o]*z**2
                                for r in range(STOnG):
                                    for s in range(len(mol)):
                                        Zd = elementos[mol[s][0]]['Carga']
                                        Cd = mol[s][1]
                                        for t in range(elementos[mol[s][0]]['Cmax']):
                                            d_vec_t = D[t]
                                            z = elementos[mol[s][0]]['Z']
                                            α_vec_t = α[t]*z**2
                                            for u in range(STOnG):
                                                c = (l+1)*(o+1)-1
                                                d = (s+1)*(t+1)-1
                                                tensor_multi_electronico[a,b,c,d] += d_vec_m[p]*d_vec_n[q]*d_vec_o[r]*d_vec_t[u]*(
                                                multi((α_vec_m[p],Ca),(α_vec_n[q],Cb),(α_vec_o[r],Cc),(α_vec_t[u],Cd)))

Escribe La molécula a calcular 'H2' o 'HeH':HeH


In [5]:
Hcore = T + V

In [6]:
# Ortogonalización de la base

evalS, U = linalg.eig(S)
diagS = dot(U.T,dot(S,U))
diagS_Mitad = diag(diagonal(diagS)**-0.5)
X = dot(U,dot(diagS_Mitad,U.T))

In [7]:
def Elementos_de_matriz_de_densidad(Ptilde, P):
    x = 0
    for i in range(B):
        for j in range(B):
            x += B**-2*(Ptilde[i,j]-P[i,j])**2
    return x**0.5

In [8]:
# Iniciamos la matriz P que será la que nos dará resultados
P = zeros((B,B))
P_ant = zeros((B,B))
P_lista = []

#Proceso iterativo
umbral = 100
while umbral > 1E-4:
    G = zeros((B,B))
    for i in range(B):
        for j in range(B):
            for x in range(B):
                for y in range(B):
                    G[i,j] += P[x,y]*(tensor_multi_electronico[i,j,y,x]-0.5*tensor_multi_electronico[i,x,y,j])
                    
    Fock = Hcore + G
    
    #Calculemos la matriz de Fock en la base ortogonal
    Fockᵪ = dot(X.T,dot(Fock,X))
    evalFockᵪ, Cᵪ = linalg.eig(Fockᵪ)
    
    #Reacomodar los eigenvalores y vectores, empezando del orbital base
    idx = evalFockᵪ.argsort()
    evalFockᵪ = evalFockᵪ[idx]
    Cᵪ = Cᵪ[:,idx]
    C = dot(X,Cᵪ)
    
    #Generar la nueva propuesta a soluciónm 'P'
    for i in range(B):
        for j in range(B):
            for a in range(int(N/2)):
                P[i,j] = 2*C[i,a]*C[j,a]
    
    P_lista.append(P)
    umbral = Elementos_de_matriz_de_densidad(P_ant,P)
    P_ant = P.copy()

print('Se logró convergencia a las {} iteraciones'.format(len(P_lista)),'\n')
print('La energia obtenida es: {} y {} Hartrees\n \t\t     o: {} y {} eV'.format(evalFockᵪ[0],evalFockᵪ[1],evalFockᵪ[0]*27.2114,evalFockᵪ[1]*27.2114))

Se logró convergencia a las 6 iteraciones 

La energia obtenida es: -1.5974477442856965 y -0.061669520372867936 Hartrees
 		     o: -43.4687895488558 y -1.6781139866742587 eV
